In [1]:
# Import the necessary packages
from google.colab import drive
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from sklearn.svm import SVR 
from imutils import paths
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import skimage.io
import skimage.color
import skimage.filters
import tensorflow as tf
import argparse


ModuleNotFoundError: No module named 'google.colab'

Mounting and defining global variable path

In [ ]:
drive.mount('/content/gdrive')
image_path = "/content/gdrive/My Drive/flash_off_elepahntOneToEight"

Reading image

In [ ]:
path=image_path
img = cv2.imread(path)

: 

Testing 3 thresholds:

In [ ]:
# read the image
img = cv2.imread("/content/gdrive/My Drive/IMG_0014.jpg")
x=int(img.shape[0]/5)
y=int(img.shape[1]/5)
img = cv2.resize(img, (x, y), interpolation=cv2.INTER_NEAREST)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


# apply simple thresholding with a hardcoded threshold value
(T, threshInv) = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV)

print('simple thresholding ')
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(threshInv, cv2.COLOR_BGR2RGB))


# binarize the image
binr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
 
print('binarized automatic thresholding')
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(binr, cv2.COLOR_BGR2RGB))

mask = cv2.adaptiveThreshold(gray, 
                              255, 
                              cv2.ADAPTIVE_THRESH_MEAN_C, 
                              cv2.THRESH_BINARY, 
                              10001, 
                              70
)

print('adaptiveThreshold')
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))

# **Moment invariant**

Reading image file

In [11]:
img1 = cv2.imread("Segmented_Teeth/segmented tooth1_1.png")
img2 = cv2.imread("Segmented_Teeth/segmented tooth1_2.png")
img3 = cv2.imread("Segmented_Teeth/segmented_tooth1_3.png")

In [12]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

In [13]:
moments1 = cv2.HuMoments(cv2.moments(img1)).flatten()
print("ORIGINAL MOMENTS for img 1: {}".format(moments1))

moments2 = cv2.HuMoments(cv2.moments(img2)).flatten()
print("ORIGINAL MOMENTS for img 2: {}".format(moments2))

moments3 = cv2.HuMoments(cv2.moments(img3)).flatten()
print("ORIGINAL MOMENTS for img 3: {}".format(moments3))

ORIGINAL MOMENTS for img 1: [9.31673872e-04 3.94367501e-07 1.16589393e-10 1.50416801e-11
 6.29560476e-22 9.44115823e-15 2.07860839e-23]
ORIGINAL MOMENTS for img 2: [9.32487012e-04 4.19574583e-07 7.85269033e-11 1.14095140e-11
 3.37913841e-22 7.05015323e-15 4.94653700e-23]
ORIGINAL MOMENTS for img 3: [1.18052474e-03 9.45728897e-07 1.96083982e-10 8.24694512e-11
 1.04041240e-20 7.77455385e-14 1.31763320e-21]


In [14]:
# for i in range(0,7):
#    moments1[i] = -np.sign(1.0, moments1[i])*np.math.log10(np.abs(moments1[i]))
   
#-1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i]))

lmoment1 = np.abs(np.log10(moments1))
print('log transformation for moments1',lmoment1)
lmoment2 = np.abs(np.log10(moments2))
print('log transformation for moments2', lmoment2)
lmoment3 = np.abs(np.log10(moments3))
print('log transformation for moments3', lmoment3)

log transformation for moments1 [ 3.03073608  6.40409888  9.93334096 10.82270365 21.20096254 14.02497472
 22.68222732]
log transformation for moments2 [ 3.03035721  6.37719083 10.10498153 10.94273285 21.47119402 14.15180144
 22.30569874]
log transformation for moments3 [ 2.92792491  6.02423334  9.70755788 10.0837069  19.98279448 13.10932452
 20.88020547]


In [15]:
dist = np.linalg.norm(lmoment1 - lmoment2)
print('Distance between moment1 and moment2',dist)

Distance between moment1 and moment2 0.5248576217835638


In [17]:
dist = np.linalg.norm(lmoment1 - lmoment3)
print('Distance between moment1 and moment3',dist)

Distance between moment1 and moment3 2.5142779577407346


# *Data Augmentation*

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import argparse
from imutils import paths
import cv2


In [6]:
datapath="training dataset/crocodiles"
out="training dataset/crocodiles_aug"

datapath1="training dataset/sharks"
out1="training dataset/sharks_aug"

In [7]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(datapath))
imagePaths1 = list(paths.list_images(datapath1))
data=[]
data1=[]

[INFO] loading images...


In [8]:
for imagePath in imagePaths:
    print(imagePath)
    image = cv2.imread(imagePath)
	#cv2.imshow(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    data.append(image)
datanp=np.array(data)

for imagePath1 in imagePaths1:
    print(imagePath1)
    image1 = cv2.imread(imagePath1)
	#cv2.imshow(image)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image1 = cv2.resize(image1, (224, 224))
    image1 = img_to_array(image1)
    image1 = np.expand_dims(image1, axis=0)
    data1.append(image1)
datanp1=np.array(data1)

training dataset/crocodiles\17095-7.jpg
training dataset/crocodiles\alligator-crocodile-fossils-5.jpg
training dataset/crocodiles\alligator-crocodile-fossils-gator-tooth.jpg
training dataset/crocodiles\cretaceous-crocodile-tooth-from-morocco-sku-v1725-elosuchus-cherifiensis-p1785-4765_image.jpg
training dataset/crocodiles\crocodile-tooth-miocene-fossil_1_bc5be32c6b8c0b8c4e3650ffe3deadd4.jpg
training dataset/crocodiles\images (1).jpg
training dataset/crocodiles\images.jpg
training dataset/crocodiles\lf (1).jpg
training dataset/crocodiles\lf.jpg
training dataset/crocodiles\miocene-fossil-crocodile-tooth-from-bone-valley-phosphates-polk-county-florida-usa-sku-v3089-gavialosuchus-americanus-p6069-17444_zoom.jpg
training dataset/crocodiles\segmented tooth3_1.JPG
training dataset/crocodiles\segmented tooth3_2.JPG
training dataset/crocodiles\segmented tooth3_3.JPG
training dataset/crocodiles\segmented tooth3_4.JPG
training dataset/crocodiles\segmented_tooth2_1.JPG
training dataset/crocodiles\

In [9]:
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
total = 0
imageGen=[]

aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
total = 0
imageGen1=[]

In [10]:
print("[INFO] generating images...")

# augmentedImages=[]
for i in range(len(datanp)):
        print("Generating is running")
  
        imageGen= aug.flow(datanp[i], batch_size=1, save_to_dir=out,
        save_prefix="image", save_format="jpg")
        for image in imageGen:
	# increment our counter
	        total += 1
	# if we have reached the specified number of examples, break
	# from the loop
	        if (total%6==0):
				#total = 0
		        break

# augmentedImages=[]
for i in range(len(datanp1)):
        print("Generating is running")
  
        imageGen1= aug.flow(datanp1[i], batch_size=1, save_to_dir=out1,
        save_prefix="image", save_format="jpg")
        for image1 in imageGen1:
	# increment our counter
	        total += 1
	# if we have reached the specified number of examples, break
	# from the loop
	        if (total%6==0):
				#total = 0
		        break

[INFO] generating images...
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Generating is running
Gene

# *CNN classification*

In [11]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(2,activation = "softmax")   #Adding the Output Layer
])

In [13]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [14]:
bs=30         #Setting batch size
train_dir = "training dataset"   #Setting training directory
validation_dir = "validation dataset"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))

Found 347 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [15]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    # steps_per_epoch=150 // bs,
                    epochs=10,
                    # validation_steps=50 // bs,
                    verbose=2)

Epoch 1/10
12/12 - 9s - loss: 0.6528 - acc: 0.6110 - val_loss: 0.6454 - val_acc: 0.5000 - 9s/epoch - 776ms/step
Epoch 2/10
12/12 - 5s - loss: 0.5426 - acc: 0.6830 - val_loss: 0.5943 - val_acc: 0.7857 - 5s/epoch - 440ms/step
Epoch 3/10
12/12 - 5s - loss: 0.4913 - acc: 0.8098 - val_loss: 0.4913 - val_acc: 0.7857 - 5s/epoch - 421ms/step
Epoch 4/10
12/12 - 5s - loss: 0.3131 - acc: 0.8761 - val_loss: 0.3467 - val_acc: 0.7857 - 5s/epoch - 412ms/step
Epoch 5/10
12/12 - 5s - loss: 0.4419 - acc: 0.7695 - val_loss: 0.3864 - val_acc: 0.8571 - 5s/epoch - 431ms/step
Epoch 6/10
12/12 - 5s - loss: 0.3790 - acc: 0.8012 - val_loss: 0.3480 - val_acc: 0.7857 - 5s/epoch - 421ms/step
Epoch 7/10
12/12 - 5s - loss: 0.2715 - acc: 0.9020 - val_loss: 0.5556 - val_acc: 0.7857 - 5s/epoch - 413ms/step
Epoch 8/10
12/12 - 5s - loss: 0.2287 - acc: 0.9193 - val_loss: 0.4303 - val_acc: 0.7143 - 5s/epoch - 440ms/step
Epoch 9/10
12/12 - 5s - loss: 0.1981 - acc: 0.9107 - val_loss: 0.3760 - val_acc: 0.7857 - 5s/epoch - 438